# Create SDS Data

In [1]:
%matplotlib inline

import datetime as dt
import itertools
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()

tqdm.pandas()

def isnum(x):
    if x is None:
        return False
    try:
        float(x)
        return True
    except ValueError:
        return False

Welcome, Abhishek Pratap!



In [4]:
v1sid, v2sid = 'syn12181339', 'syn12181348'

v1r = pd.read_csv(syn.get(v1sid).path, parse_dates=['timestamp'])
v2r = pd.read_csv(syn.get(v2sid).path, parse_dates=['timestamp'])

### Process V1 Data

In [5]:
v1 = v1r.drop(columns=['userid','timestampUTC']) \
    .rename(columns={
        'timestamp':'dt_response',
        'brightenid':'participant_id'
})

In [ ]:
v1.support.hist()

### Process V2 data

In [ ]:
v2 = v2r.drop(columns='userid').rename(columns={
    'brightenid':'participant_id',
    'timestamp':'dt_response'
})

v2.support = v2.support/10

In [ ]:
v2.support.hist()

### Combine the DataFrames

In [ ]:
combined = pd.concat([v1, v2], sort=False)
combined.head()

In [ ]:
len(combined.loc[combined.participant_id.isnull()])/len(combined)

In [ ]:
combined = combined.dropna(subset=['participant_id'], how='any')

### Add week into study

In [6]:
metasid = 'syn27082597'

In [ ]:
metasid = 'syn27082597'
metadata = syn.tableQuery(f'SELECT participant_id, startdate FROM {metasid}').asDataFrame(convert_to_datetime=True)
metadata.startdate = pd.to_datetime(metadata.startdate)

# add in the participants start date as a new column
combined = pd.merge(combined, metadata, on='participant_id', how='left')

# get the time difference in weeks as a float
combined['week'] = [
    d.days/7 for d in (
        combined.dt_response.apply(
            lambda x: dt.datetime(year=x.year, month=x.month, day=x.day))-combined.startdate
    )
]

# convert the week number to an int by taking the floor
combined.week = combined.week.progress_apply(lambda x: np.int16(np.floor(x))+1 if not pd.isnull(x) else np.nan)

# remove the start date
combined = combined.drop(columns=['startdate'], errors='ignore')

# reorder the columns
cols = list(combined.columns)
cols = cols[0:2] + ['week'] + cols[2:-1]
combined = combined.reindex(columns=cols)

combined.head()

### Localize timestamps

In [ ]:
# localize timestamps
combined['dt_response'] = [
    str(t.tz_localize('UTC'))
    for t in combined.dt_response
]

combined.head()

### Set provenance and upload to Synapse

In [ ]:
t = syn.delete(
    syn.tableQuery('select * from syn17022658')
)

In [ ]:
final = syn.store(Table(
    Schema(
            name='SDS',
            columns=as_table_columns(combined), 
            parent='syn10848316'),
        combined
    )
)

In [7]:
final = syn.setProvenance(
    'syn17022658',
    activity=Activity(
        name='Combine V1 and V2 data',
        description='Process and combine the data collected during study 1 and study 2',
        used=[v1sid, v2sid, metasid],
        executed=[
            dict(
                name='Create_SDS_data',
                url='https://github.com/apratap/BRIGHTEN-Data-Release/blob/master/Create_SDS_data.ipynb'
            )
        ]
    )
)